In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import os
# os.chdir("../")


In [2]:
!pwd

/c/Users/hakos/GDrive/Downloads/Research/data/yakuji


In [3]:
def last_day_of_month(row):
    if row['month'] == 12:
        next_month = 1
        next_year = row['year'] + 1
    else:
        next_month = row['month'] + 1
        next_year = row['year']
    last_day = datetime.datetime(next_year, next_month, 1) - datetime.timedelta(days=1)
    return last_day

In [4]:
data=pd.read_csv("../merged/len3_ship_mon.csv",encoding="shift-jis",index_col=0)
count=0
# drop never treated group
data=data.dropna(subset=["release_year"])
data["r_date"]=pd.to_datetime(data["date"],format="%Y/%m/%d")
data["r_date"]

36      1995-01-24
37      1995-02-10
38      1995-01-24
39      1995-02-10
40      1995-01-24
           ...    
39756   2011-05-10
39869   2001-09-10
39870   2001-09-10
39871   2001-09-10
39872   2001-09-10
Name: r_date, Length: 16168, dtype: datetime64[ns]

In [5]:
data["ship_date"]=data.apply(lambda row: last_day_of_month(row),axis=1)

1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [6]:
data["elapsed"]=data["ship_date"]-data["r_date"]
data["elapsed"]=data["elapsed"].dt.days//91
data["elapsed"]

36       56
37       56
38       56
39       56
40       56
         ..
39756    48
39869    85
39870    86
39871    86
39872    86
Name: elapsed, Length: 16168, dtype: int64

In [7]:
# data.loc[(data['elapsed'] > 10) | (data["elapsed"]< -5), 'elapsed'] = np.nan
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
if count:
    # count ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"month"]).first(), 
        dummies.groupby([data['year'],data['id'],data["month"]]).sum()
    ], axis=1).reset_index()
else: 
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id',"month"]).first(), 
        dummies.groupby([data['year'],data['id'],data["month"]]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
aggregated_df

,year,id,month,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,...,elapsed_124,elapsed_125,elapsed_126,elapsed_127,elapsed_128,elapsed_129,elapsed_130,elapsed_131,elapsed_132,elapsed_133
0,2009,114,1,解熱鎮痛消炎剤,9028279,1178981,0.0,17139194,2108808,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,6285471,1981401,0.0,18952804,2554201,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,9085356,2726852,0.0,24380772,2372946,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,11595639,2548806,0.0,23724036,2409430,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,8564391,2790706,0.0,22691023,2110536,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4467,2023,629,4,その他の化学療法剤,5676488,-,0.0,17192413,-,0.0,...,0,0,0,0,0,0,0,0,0,0
4468,2023,799,1,他に分類されない治療を主目的としない医薬品,1124555,234387,0.0,1283795,204783,0.0,...,0,0,0,0,0,0,0,0,0,0
4469,2023,799,2,他に分類されない治療を主目的としない医薬品,1299090,285720,0.0,1378071,167623,0.0,...,0,0,0,0,0,0,0,0,0,0
4470,2023,799,3,他に分類されない治療を主目的としない医薬品,1547593,577328,0.0,1497563,170752,0.0,...,0,0,0,0,0,0,0,0,0,0


connect generic usage

In [8]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len3_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric

,薬効分類,year,generic_share_q,generic_share_r
0,114.0,2008.0,0.223357,0.058586
1,114.0,2009.0,0.223357,0.058586
2,114.0,2010.0,0.223357,0.058586
3,114.0,2011.0,0.223357,0.058586
4,114.0,2012.0,0.227087,0.058586
...,...,...,...,...
395,799.0,2019.0,0.006196,0.024777
396,799.0,2020.0,0.006056,0.022847
397,799.0,2021.0,0.007953,0.028184
398,799.0,2022.0,0.028110,0.043253


In [9]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["薬効分類","year"])
merged_df

,year,id,month,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,...,elapsed_129,elapsed_130,elapsed_131,elapsed_132,elapsed_133,Year,Percentage,薬効分類,generic_share_q,generic_share_r
0,2009,114,1,解熱鎮痛消炎剤,9028279,1178981,0.0,17139194,2108808,0.0,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
1,2009,114,2,解熱鎮痛消炎剤,6285471,1981401,0.0,18952804,2554201,0.0,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
2,2009,114,3,解熱鎮痛消炎剤,9085356,2726852,0.0,24380772,2372946,0.0,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
3,2009,114,4,解熱鎮痛消炎剤,11595639,2548806,0.0,23724036,2409430,0.0,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
4,2009,114,5,解熱鎮痛消炎剤,8564391,2790706,0.0,22691023,2110536,0.0,...,0,0,0,0,0,2009,35.8,114.0,0.223357,0.058586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,2022,799,8,他に分類されない治療を主目的としない医薬品,1500193,305986,0.0,1739639,384543,0.0,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253
4196,2022,799,9,他に分類されない治療を主目的としない医薬品,1572585,436842,0.0,2150410,340113,0.0,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253
4197,2022,799,10,他に分類されない治療を主目的としない医薬品,1523822,445750,0.0,1253828,311956,0.0,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253
4198,2022,799,11,他に分類されない治療を主目的としない医薬品,1471887,210242,0.0,1204241,310745,0.0,...,0,0,0,0,0,2022,79.0,799.0,0.028110,0.043253


In [10]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,month,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,...,id_264,id_265,id_332,id_339,id_399,id_441,id_449,id_625,id_629,id_799
0,2009,114,1,解熱鎮痛消炎剤,9028279,1178981,0.0,17139194,2108808,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,6285471,1981401,0.0,18952804,2554201,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,9085356,2726852,0.0,24380772,2372946,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,11595639,2548806,0.0,23724036,2409430,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,8564391,2790706,0.0,22691023,2110536,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,2022,799,8,他に分類されない治療を主目的としない医薬品,1500193,305986,0.0,1739639,384543,0.0,...,0,0,0,0,0,0,0,0,0,1
4196,2022,799,9,他に分類されない治療を主目的としない医薬品,1572585,436842,0.0,2150410,340113,0.0,...,0,0,0,0,0,0,0,0,0,1
4197,2022,799,10,他に分類されない治療を主目的としない医薬品,1523822,445750,0.0,1253828,311956,0.0,...,0,0,0,0,0,0,0,0,0,1
4198,2022,799,11,他に分類されない治療を主目的としない医薬品,1471887,210242,0.0,1204241,310745,0.0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if not(count):
    print("dummy")
    # dummy ver.
    df_with_dummies.to_csv("../merged/len3_ship_mon_Q_DN.csv",encoding="shift-jis")
else:    
    # count ver.
    df_with_dummies.to_csv("../merged/len3_ship_mon_Q_sum_DN.csv",encoding="shift-jis")

dummy
